# Armani SageMaker instance 

In [15]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [16]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [17]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "./products.csv")
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv', index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


Now that we have loaded into model_data the CSV...

In [18]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

               1                             2                              3   
0                                                                               
donna    giacche                           blu                           lino  \
donna  pantaloni  grigio scuro, ghiaccio e blu                           lino   
donna  pantaloni                      fantasia                  twill di seta   
donna   sneakers                  bianco e blu                          pelle   
donna      abiti                    nero e blu                         jersey   
donna    giacche                      fantasia                  twill di seta   
donna    sandali                 argento e oro                          pelle   
donna    sandali                          nero                        vernice   
donna    sandali                nero e marrone                          nappa   
donna    sandali                          nero                   raso sautage   
donna    giacche            

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [19]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(35, 13) (15, 13)


We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [20]:
print(train_data)
print(test_data)

               1                            2                              3   
0                                                                              
uomo     giacche                          blu                   misto cotone  \
uomo   pantaloni                     fantasia                          cupro   
donna      abiti                   blu e nero                  misto viscosa   
uomo       abiti              nero e blu navy                   lana vergine   
uomo   pantaloni                        beige                   lana vergine   
donna     scarpe                  nude e nero                          pelle   
uomo     giacche                        beige  viscosa, poliammide e elastan   
donna    sandali                          oro                 viscosa e seta   
uomo       abiti                         nero                   lana vergine   
donna     scarpe                  nude e nero                          pelle   
donna    giacche                        

In [21]:
train_data.to_csv(
    'train.csv',
    index=False,
    header=True
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [22]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(
    xgboost_container,role, instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=f's3://{bucket_name}/{prefix}/output',
    sagemaker_session=sess
)
xgb.set_hyperparameters(
    max_depth=5,eta=0.2,gamma=4,
    min_child_weight=6,subsample=0.8,
    silent=0,objective='binary:logistic',
    num_round=100
)

In [23]:
print(s3_input_train)

In [24]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-22-13-29-20-879


2023-06-22 13:29:21 Starting - Starting the training job...
2023-06-22 13:29:36 Starting - Preparing the instances for training......
2023-06-22 13:30:30 Downloading - Downloading input data...
2023-06-22 13:31:11 Training - Training image download completed. Training in progress...Arguments: train
[2023-06-22:13:31:29:INFO] Running standalone xgboost training.
[2023-06-22:13:31:29:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-22:13:31:29:INFO] File size need to be processed in the node: 0.03mb. Available memory size in the node: 55595.48mb
[2023-06-22:13:31:29:INFO] Determined delimiter of CSV input is ','
[13:31:29] S3DistributionType set as FullyReplicated
[2023-06-22:13:31:29:ERROR] Customer Error: Failed to load csv data with exception:
b"[13:31:29] src/data/csv_parser.h:124: Deduced delimiter ',' is not found in the line. Expected ',' as the delimiter to separate fields.\n\nStack trace returned 7 entries:\n[bt] (0) /usr/local/lib/python3.7/site-packages/xgboos

UnexpectedStatusException: Error for Training job xgboost-2023-06-22-13-29-20-879: Failed. Reason: ClientError: Failed to load csv data with exception:
b"[13:31:29] src/data/csv_parser.h:124: Deduced delimiter ',' is not found in the line. Expected ',' as the delimiter to separate fields.\n\nStack trace returned 7 entries:\n[bt] (0) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::StackTrace[abi:cxx11]()+0x3b) [0x7f22006bebbb]\n[bt] (1) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x29) [0x7f22006bf2f9]\n[bt] (2) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::data::CSVParser<unsigned int, float>::ParseBlock(char const*, char const*, dmlc::data::RowBlockContainer<unsigned int, float>*)+0x3f9) [0x7f220087ce99]\n[bt] (3) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(+0x21d823) [0x7f2200862